In [16]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [17]:
load_dotenv(override=True) #Why ?

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"Bạn đã đăng nhập được khóa với mã {openai_api_key[:8]}")
else:
    print("Không tìm thấy mã khóa")

MODEL = "gpt-4.1-mini"
openai = OpenAI()

Bạn đã đăng nhập được khóa với mã sk-proj-


In [ ]:
system_message = """
Bạn là một hệ thống hỗ trợ bán hàng. 
Nhiệm vụ của bạn là thông báo cho khách hàng những sản phầm có bên trong dữ liệu, và hãy trả lời tôi không biết khi bạn không biết thông tin gì về món hàng đó. 
Không trả lời những câu hỏi ngoài khía cạnh buôn bán của cửa hàng. Tất cả các giá khi được khách hàng nhắc đến đều quy về tiền VNĐ hết " 
"""

In [19]:
# Kiểm tra cơ bản hệ thống
def basic_chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": "system_message"}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages
    )
    return response.choices[0].message.content

In [20]:
gr.ChatInterface(fn=basic_chat).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [21]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_products",
    "description": "Lấy giá và trả về tên quần áo .",
    "parameters": {
        "type": "object",
        "properties": {
            "name_of_product": {
                "type": "string",
                "description": "Cái quần áo cái mà khách hàng muốn",
            },
        },
        "required": ["name_of_product"],
        "additionalProperties": False
    }
}



In [22]:
#Dummy data
products = {
    "áo thun": "50.000",
    "quần tây": "70.000",
    "áo khoác": "100.000",
    "quần thể thao": "80.000"
}

def get_products (name_of_product):
    print(f"Bạn đang lấy sản phẩm {name_of_product}")
    price = products.get(name_of_product.lower(), f"Tôi không có sản phẩm này")
    return f"Giá của sản phẩm {name_of_product} là {price}"

print(get_products(f"Quần tây"))

Bạn đang lấy sản phẩm Quần tây
Giá của sản phẩm Quần tây là 70.000


In [23]:
tools = [{"type": "function", "function": price_function}]

tools

[{'type': 'function',
  'function': {'name': 'get_products',
   'description': 'Lấy giá và trả về tên quần áo .',
   'parameters': {'type': 'object',
    'properties': {'name_of_product': {'type': 'string',
      'description': 'Cái quần áo cái mà khách hàng muốn'}},
    'required': ['name_of_product'],
    'additionalProperties': False}}}]

In [24]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_products":
        arguments = json.loads(tool_call.function.arguments)
        product = arguments.get('name_of_product')
        price_details = get_products(product)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [25]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": "system_message"}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages
    )

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [26]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [27]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    responses = []
    tool_call = message.tool_calls[0]
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_products":
            arguments = json.loads(tool_call.function.arguments)
            product = arguments.get('name_of_product')
            price_details = get_products(product)
            responses.append ({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [28]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": "system_message"}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages
    )

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [32]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
